In [71]:
import numpy as np
import re

train_data=np.load("data_train.pkl",allow_pickle=True)
test_data=np.load("data_test.pkl",allow_pickle=True)
train_input=np.array(train_data)[0]
train_label=np.array(train_data)[1]
train_input_10=train_input[:60000]
train_label_10=train_label[:60000]
test_input=np.array(test_data)
test_input_10=train_input[-10000:]
test_label_10=train_label[-10000:]

labels = ['anime', 'nfl', 'soccer', 'canada', 'Overwatch', 'leagueoflegends', 'europe', 'worldnews', 'wow', 'Music', 'nba', 'funny', 'movies', 'baseball', 'hockey', 'GlobalOffensive', 'trees', 'gameofthrones', 'conspiracy', 'AskReddit']

#creating bag of words for the training set

import scipy.sparse as sp

import nltk


from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

#scanning through training set to collect all words 
unique_words = []
for doc in train_input_10: 
    doc = re.sub("[^0-9a-zA-Z]+", ' ', doc).strip() #removing special characters
    for word in doc.split(" "):
        if word not in stopWords:
            unique_words.append(word) 
        
unique_words=list(set(unique_words)) #returns a list of unique words

bow=sp.csr_matrix((0,len(unique_words)))
bow_test=sp.csr_matrix((0,len(unique_words)))

#creating bag of words for the training set
dictionary={word:i for i,word in enumerate(unique_words)}

for doc in train_input_10:
    doc_features=np.zeros(len(unique_words)) #initializing 
    doc = re.sub("[^0-9a-zA-Z]+", ' ', doc).strip()#removing special characters
    for word in doc.split(" "):
        if word not in stopWords:
            doc_features[dictionary[word]]+=1
    bow=sp.vstack((bow,sp.csr_matrix(doc_features)))
    
    
#creating bag of words for the testing set 

for doc in test_input_10:
    doc = re.sub("[^0-9a-zA-Z]+", ' ', doc).strip() #removing special characters
    doc_features=np.zeros(len(unique_words)) #initializing 
    for word in doc.split(" "):
        if word in dictionary:
            doc_features[dictionary[word]]+=1
    bow_test=sp.vstack((bow_test,sp.csr_matrix(doc_features)))

    #bow_test=sp.csr_matrix.toarray(bow_test)
bow_test=bow_test.A

#computing priors

doc_locations=[]
prior=np.zeros(len(labels))
for i,elem in enumerate(labels):
    elem_index=np.where(train_label_10==elem)
    doc_locations.append(elem_index[0])
    elem_len =len(elem_index[0])
    elem_prior=elem_len/train_label_10.shape[0]
    prior[i]=elem_prior


In [79]:
#computing conditional probability, multinomial 

alpha_group=[0.2657]

alpha_group_pred_rate=[]

for i,elem in enumerate(alpha_group):
    alpha=alpha_group[i]

    cond_prob=[]
    for i in doc_locations:
        word_freq_per_class=np.sum(bow[i],axis=0)
        total_word_per_class=np.sum(word_freq_per_class)
        cond_prob_given_a_class=(word_freq_per_class+alpha)/(total_word_per_class+alpha*len(unique_words))
        cond_prob.append(cond_prob_given_a_class)    

    cond_prob=np.array(cond_prob)
    
#calculating multinomial prob for each testing sample each class, and return predicted label
    pred=[]
    for j,input_elem in enumerate(test_input_10): #loop over each test sample
        prob_per_class=[]
        for i,elem in enumerate(labels): #loop over 20 labels to compute prob given a class for each sample
            prob=np.power(cond_prob[i],bow_test[j])
            log_prob=np.log(prior[i])+np.sum(np.log(prob))
            prob_per_class.append(log_prob)
        label=np.argmax(prob_per_class)
        pred.append(labels[label])

    alpha_group_pred_rate.append(np.mean(pred == test_label_10, axis=0))
#print(pred)
print(alpha_group, alpha_group_pred_rate)


#export predicitons into csv file
#import csv
#np.savetxt('phase_1_pred.csv',np.array(pred),delimiter='\n',newline='\n',fmt='%s')

[0.2655, 0.2656, 0.2657, 0.2658, 0.2659] [0.5511, 0.5511, 0.5511, 0.5511, 0.5511]
